###Get access token by OAuth2

In [10]:
import requests
from requests_oauthlib import OAuth2Session
import json
import pandas as pd

client_id = "androidClient"
client_secret = "secret"
lines = [line.rstrip('\n') for line in open('pass.txt')]
username = lines[0]
password = lines[1]
dev_token = lines[2]

authorization_url = "https://app.atimelogger.com/oauth/token?username={0}&password={1}&grant_type=password".format(username, password)
token = requests.post(authorization_url, auth=(client_id, client_secret))
access_token = json.loads(token.text)['access_token']

### Reference Dictionary
Get a dictionary contain types' **ID, Name, Parent's name** for reference.

In [11]:
r_type = requests.get("https://app.atimelogger.com/api/v2/types", 
                 headers = {'Authorization': "bearer " + access_token})
json_type = json.loads(r_type.text)
# for d in json_type['types']:
#     if d['parent']:
#         d['parent'] = d['parent']['guid']
# data = pd.DataFrame(json_type["types"])
# types = pd.DataFrame([d for d in json_type['types'] if not d['group']]).ix[:, [4,5,2,6]]
# group = pd.DataFrame([d for d in json_type['types'] if d['group']]).ix[:, [4,5,2]]
# pd.merge(types, group, left_on = 'parent', right_on = 'guid', how = 'left')

dict_group = {d['guid']:d['name'] for d in json_type['types'] if d['group']}
dict_type = {}
dict_id = {}
for d in json_type['types']:
    if not d['group']:
        parent_name = dict_group[d['parent']['guid']] if d['parent'] != None else None
        dict_type[d['guid']] = (d['name'], parent_name)
        dict_id[d['name']] = (d['guid'], parent_name)

###Helper Function

In [12]:
from datetime import datetime, date, timedelta
def getTime(unixSecond):
    """"
    unixSecond: unix timestamp
    returns: string format of date time for unix timestamp
    """
    return str(datetime.fromtimestamp(unixSecond))

def getDelta(duration):
    """"
    duration: duration in second
    returns: string format of duration
    """
    return str(timedelta(seconds = duration)).split('.')[0]

def getThisWeek(past = False):
    """"
    past: whether the week want has past or not, if 'True', get last week
    returns: tuple of begin and end of the week in seconds (integer)
    """
    today = date.today()
    if(past):
        d = timedelta(today.weekday())
        end = today - d
        begin = end - timedelta(7)
    else:
        end = today + timedelta(1)
        d = timedelta(today.weekday())
        begin = today - d
    begin_s = int(begin.strftime("%s"))
    end_s = int(end.strftime("%s"))
    return (begin_s, end_s)

### Today's Report
Get `today`, `yesterday` as parameters and fetch all intervals of yesterday.

In [14]:
from datetime import datetime, date, timedelta
import copy

today = date.today()
day_delta = timedelta(1)
yesterday = today - day_delta
## Following method will generate 4-hours difference with actual time
# today_s = (today - date(1970,1,1)).total_seconds()
# yesterday_s = (yesterday - date(1970,1,1)).total_seconds()
today_s = today.strftime("%s")
yesterday_s = yesterday.strftime("%s")
params = {'from': int(yesterday_s), 'to': int(today_s)}

r_today = requests.get("https://app.atimelogger.com/api/v2/intervals", 
                 params = params, headers = {'Authorization': "bearer " + access_token})
intervals = json.loads(r_today.text)['intervals']

### Test the correctness of time
interval_time = copy.deepcopy(intervals)
for d in interval_time:
    d['from'] = str(datetime.fromtimestamp(d['from']))
    d['to'] = str(datetime.fromtimestamp(d['to']))
    d['type'] = dict_type[d['type']['guid']]
pd.DataFrame(interval_time).ix[:,[2,4,5,1]].head(6)

,from,to,type,comment
0,2015-08-16 23:56:39,2015-08-17 08:36:40,"(Sleep, Health)",
1,2015-08-16 22:54:56,2015-08-16 23:56:39,"(Programming, Learn)",Time analysis
2,2015-08-16 22:40:12,2015-08-16 22:54:56,"(Restroom, Health)",
3,2015-08-16 22:39:13,2015-08-16 22:40:12,"(Rest, Health)",
4,2015-08-16 22:07:26,2015-08-16 22:39:13,"(Interview, Career)",
5,2015-08-16 22:03:51,2015-08-16 22:07:26,"(Meal, Health)",


Extract `comment` as task and calculate its time.

In [45]:
from collections import defaultdict

task = defaultdict(dict)
for d in intervals:
    if d['comment']:
        task[d['comment']]['Group'] = dict_type[d['type']['guid']][1]
        if "Time" in task[d['comment']].keys():
            task[d['comment']]['Time'] += (d['to'] - d['from'])
            task[d['comment']]['N'] += 1
        else:
            task[d['comment']]['Time'] = (d['to'] - d['from'])
            task[d['comment']]['N'] = 1

# task_df = []
# for k, v in task.items():
#     task_df.append(dict(Task = k, Group = v['Group'], 
#                           Time = getDelta(v['Time']),
#                           N = v['N'],
#                          Avg = getDelta(v['Time']/v['N'])))
# pd.DataFrame(task_df)

task_list = []
for k, v in task.items():
    task_list.append([k, v['Group'], getDelta(v['Time']), str(v['N']), getDelta(v['Time']/v['N'])])
pd.DataFrame(task_list)

,0,1,2,3,4
0,Google analytics,Career,5:05:56,5,1:01:11
1,Time analysis,Learn,6:25:32,6,1:04:15
2,Regular Expressions,Learn,0:01:14,1,0:01:14


In [18]:
import numpy as np
sleep_id = dict_id['Sleep'][0]
rest_id = dict_id['Rest'][0]
meal_id = dict_id["Meal"][0]
rest = np.array([])
meal = np.array([])
for d in intervals:
    if d['type']['guid'] == sleep_id:
        sleep = [getTime(d['from']), 
                 getTime(d['to']),
                 getDelta(d['to'] - d['from'])]
    elif d['type']['guid'] == rest_id:
        rest = np.append(rest, d['to'] - d['from'])
    elif d['type']['guid'] == meal_id:
        meal = np.append(meal, d['to'] - d['from'])
print "Sleep: ", sleep
print "Total Rest Time: ", getDelta(rest.sum())
print "Total Meal Time: ", getDelta(meal.sum())

Sleep:  ['2015-08-15 00:25:22', '2015-08-15 08:58:40', '8:33:18']
Total Rest Time:  0:15:14
Total Mean Time:  2:49:11


### Import Evernote and find template

In [3]:
from evernote.api.client import EvernoteClient
client = EvernoteClient(token=dev_token, sandbox=False)
noteStore = client.get_note_store()

## search diary template
from evernote.edam.notestore.ttypes import NoteFilter, NotesMetadataResultSpec
noteStore = client.get_note_store()
search = NoteFilter()
search.words = "Daily Review Template"
search.ascending = False

spec = NotesMetadataResultSpec()
spec.includeTitle = True

ourNoteList = noteStore.findNotesMetadata(dev_token, search, 0, 100, spec)
template = noteStore.getNoteContent(dev_token, ourNoteList.notes[0].guid)

## get Diary notebook guid
notebooks = noteStore.listNotebooks()
for n in notebooks:
    if(n.name == 'Diary'):
        diary_notebook = n.guid

### Build diary

In [67]:
start = template.find("<tr>", template.find("<tr>") + 1)
end = template.find("</tr>", template.find("</tr>") + 1)
row_template = template[start:end+len("</tr>")]

rows = ""
for task in task_list:
    row = row_template
    for item in task:
        row = row.replace('<br/>', item, 1)
    rows += row
rows
diary = template[:start] + str(rows) + template[end+len('</tr>'):]
title = str(yesterday).replace("-", "")

###Create note

In [68]:
import evernote.edam.type.ttypes as Types
note = Types.Note()
note.title = title
note.content = diary
note.notebookGuid = diary_notebook
note = noteStore.createNote(note)

### Get access token by basic authorization

In [ ]:
from requests.auth import HTTPBasicAuth
import time

# r = requests.get('https://app.atimelogger.com', auth=HTTPBasicAuth(username, password))
today = datetime.date.today()
day_delta = datetime.timedelta(1)
yesterday = today - day_delta
today_s = (today - datetime.date(1970,1,1)).total_seconds()
print today_s
yesterday_s = (yesterday - datetime.date(1970,1,1)).total_seconds()
data = {'from': int(yesterday_s), 'to': int(today_s)}
r = requests.get("https://app.atimelogger.com/api/v2/intervals", params = data, auth=HTTPBasicAuth(username, password))
print(json.dumps(json.loads(r.text), indent=2))
# print int(time.time())
# print time.gmtime()